<a href="https://colab.research.google.com/github/pythonuzgit/elmurodov/blob/master/Graph%20Neural%20Network/Graph_Neural_Network_in_NLP_with_Movies_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910454 sha256=4a1173eff1c699fe25bc55024335f165c52eb9b810c6f846411825c364a455dc
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch_geometric


Import

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

Load the code search dataset

In [ ]:
df = pd.read_csv('/content/Movies_dataset.csv')

In [ ]:
df.shape

(10000, 8)

In [ ]:
df.columns

Index(['index', 'title', 'original_language', 'release_date', 'popularity',
       'vote_average', 'vote_count', 'overview'],
      dtype='object')

Select the relevant columns

In [ ]:
selected_columns = ['index', 'title', 'overview']
df = df[selected_columns]

In [ ]:
def preprocess_data(data):
    # Drop any rows with missing values
    #data.dropna(inplace=True)

    # Tokenize 'text' and 'code' columns
    vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
    text_features = vectorizer.fit_transform(data['title'])
    code_features = vectorizer.fit_transform(data['overview'])

    # Normalize features
    scaler = StandardScaler()
    text_features = scaler.fit_transform(text_features.toarray())
    code_features = scaler.fit_transform(code_features.toarray())

    # Split into features and labels
    X = np.concatenate((text_features, code_features), axis=1)
    y = data['index']

     #Split into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Create the processed DataFrame
    column_names = list(data.columns)[:-1]  # Exclude the 'task_id' column
    processed_data = pd.DataFrame({col: X_train[:, i] for i, col in enumerate(column_names)})
    processed_data['index'] = y_train

    return processed_data, X_test, y_test

In [ ]:
processed_data, X_test, y_test = preprocess_data(df)

Create graph representation

In [ ]:

def create_graph_representation(data):
    edge_index = np.array([[i, i] for i in range(len(data))]).T
    x = torch.tensor(data.drop(['index'], axis=1).values, dtype=torch.float)
    y = torch.tensor(data['index'].values, dtype=torch.long)
    return Data(x=x, edge_index=edge_index, y=y)

In [ ]:
graph_data = create_graph_representation(processed_data)

Define the GNN model

In [ ]:

class GNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

Set the device for computation

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Initialize the GNN model

In [ ]:

input_dim = graph_data.num_node_features
hidden_dim = 64
output_dim = len(processed_data['index'].unique())
model = GNNModel(input_dim, hidden_dim, output_dim).to(device)

Define the loss function and optimizer

In [ ]:

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

Define the custom collate function

In [ ]:
def collate_fn(batch):
    return batch

# Create the DataLoader
data_loader = DataLoader([graph_data], batch_size=1, shuffle=True, collate_fn=collate_fn)

Define the custom collate function

In [ ]:

def collate_fn(batch):
    x = [data.x for data in batch]
    edge_index = [data.edge_index for data in batch]
    y = [data.y for data in batch]
    return {'x': x, 'edge_index': edge_index, 'y': y}

# Convert the graph data to PyTorch DataLoader
data_loader = DataLoader([graph_data], batch_size=1, collate_fn=collate_fn)

In [ ]:
for data in data_loader:
    x = torch.tensor(data['x'][0], dtype=torch.float).to(device)
    edge_index = torch.tensor(data['edge_index'][0], dtype=torch.long).to(device)
    y = torch.tensor(data['y'][0], dtype=torch.float).to(device)

    # Normalize the target values
    y_min = torch.min(y)
    y_max = torch.max(y)
    y_normalized = (y - y_min) / (y_max - y_min)

    optimizer.zero_grad()
    out = model(x, edge_index)

    # Verify the target values range
    print(f"Min target value: {torch.min(y_normalized).item()}")
    print(f"Max target value: {torch.max(y_normalized).item()}")

    # Convert y_normalized to torch.long
    y_normalized = y_normalized.long()

    loss = criterion(out, y_normalized)
    loss.backward()
    optimizer.step()

<ipython-input-26-3cff8116b1d4>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(data['x'][0], dtype=torch.float).to(device)
<ipython-input-26-3cff8116b1d4>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(data['y'][0], dtype=torch.float).to(device)


Min target value: 0.0
Max target value: 1.0


Testing the model

In [ ]:

model.eval()
total_correct = 0
total_samples = 0

with torch.no_grad():
    for data in data_loader:
        x = torch.tensor(data['x'][0], dtype=torch.float).to(device)
        edge_index = torch.tensor(data['edge_index'][0], dtype=torch.long).to(device)
        y = torch.tensor(data['y'][0], dtype=torch.float).to(device)

        # Normalize the target values
        y_min = torch.min(y)
        y_max = torch.max(y)
        y_normalized = (y - y_min) / (y_max - y_min)

        out = model(x, edge_index)
        _, predicted = torch.max(out, dim=1)
        total_correct += (predicted == y_normalized).sum().item()
        total_samples += y_normalized.size(0)

accuracy = total_correct / total_samples
print(f"Accuracy: {accuracy * 100:.2f}%")



<ipython-input-27-906453e50160>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(data['x'][0], dtype=torch.float).to(device)
<ipython-input-27-906453e50160>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(data['y'][0], dtype=torch.float).to(device)


Accuracy: 79.84%
